In [82]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB

from sklearn import (
    linear_model, 
    ensemble,
    tree,
    decomposition, 
    naive_bayes, 
    neural_network,
    svm,
    metrics,
    preprocessing, 
    model_selection, 
    pipeline,
)
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, auc, roc_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
import pandas as pd

import utils


red_code = '\033[91m'
blue_code = '\033[94m'
green_code = '\033[92m'
yellow_code = '\033[93m'
reset_code = '\033[0m'

## Load Data 

In [15]:
def load_movies(path2data): # 1 classe par répertoire
    alltxts = [] # init vide
    labs = []
    cpt = 0
    for cl in os.listdir(path2data): # parcours des fichiers d'un répertoire
        for f in os.listdir(path2data+cl):
            txt = open(path2data+cl+'/'+f).read()
            alltxts.append(txt)
            labs.append(cpt)
        cpt+=1 # chg répertoire = cht classe
        
    return alltxts,labs

In [16]:
path = "./datasets/movies/movies1000/"
alltxts,alllabs = utils.load_movies(path)

movies_df = pd.DataFrame()
movies_df['text'] = alltxts
movies_df['label'] = alllabs


In [17]:
preprocessed_alltxts = [utils.preprocess(alltxt) for alltxt in movies_df.text]

preprocessed_movies_df = pd.DataFrame()
preprocessed_movies_df['text'] = preprocessed_alltxts
preprocessed_movies_df['label'] = alllabs

In [78]:
X_text_train, X_text_test, y_train, y_test = model_selection.train_test_split(preprocessed_movies_df['text'], preprocessed_movies_df['label'], test_size=0.2, random_state=24)

### GRID SEARCH

### Logistique regression :

In [102]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizers = [
    ('CountVectorizer', CountVectorizer()),
    ('TfidfVectorizer', TfidfVectorizer(smooth_idf=True, sublinear_tf=True))
]

param_grid = {
    'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'vectorizer__stop_words': [None, 'english'],
    'vectorizer__max_features': [1000, 5000, 10000, None],

}

for vectorizer_name, vectorizer in vectorizers:
    pipe = Pipeline([
        ('vectorizer', vectorizer),
        ('classifier', LogisticRegression())
    ])

    grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='f1_micro', n_jobs = -1, refit = True)  # Modifier les paramètres CV et scoring
    grid_search.fit(X_text_train, y_train)


    print(f"Classifier: LogisticRegression, Vectorizer: {vectorizer_name}")
    print("Best parameters found:")
    print(grid_search.best_params_)
    print("Best cross-validation f1:")
    print(grid_search.best_score_)
    print("")

    best_model = grid_search.best_estimator_

    y_pred = best_model.predict(X_text_test)
    report = metrics.classification_report(y_test, y_pred)
    print("Classification Report:")
    print(report)
    print("")


C:\Users\NeilB\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Classifier: LogisticRegression, Vectorizer: CountVectorizer
Best parameters found:
{'vectorizer__max_features': 10000, 'vectorizer__ngram_range': (1, 2), 'vectorizer__stop_words': None}
Best cross-validation f1:
0.8387500000000001

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.84      0.83       193
           1       0.85      0.83      0.84       207

    accuracy                           0.83       400
   macro avg       0.83      0.83      0.83       400
weighted avg       0.83      0.83      0.83       400


Classifier: LogisticRegression, Vectorizer: TfidfVectorizer
Best parameters found:
{'vectorizer__max_features': 10000, 'vectorizer__ngram_range': (1, 3), 'vectorizer__stop_words': None}
Best cross-validation f1:
0.8625

Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.87      0.85       193
           1       0.87      0.84      0.86       207

    accura

### SVM

In [42]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizers = [
    ('CountVectorizer', CountVectorizer()),
    ('TfidfVectorizer', TfidfVectorizer(sublinear_tf=True))
]

param_grid = {
    'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'vectorizer__stop_words': [None, 'english'],
    'vectorizer__max_features': [1000, 5000, 10000, None],
    'classifier__C': [0.1, 1, 10],
    'classifier__kernel': ['linear', 'rbf'],
    'classifier__gamma': [0.1, 0.01],
    'classifier__degree': [2, 3],
    'classifier__class_weight': [None, 'balanced'],
    'classifier__probability': [False, True]
}

for vectorizer_name, vectorizer in vectorizers:
    pipe = Pipeline([
        ('vectorizer', vectorizer),
        ('classifier', SVC())
    ])

    grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='f1_micro', n_jobs = -1, refit = True)  # Modifier les paramètres CV et scoring
    grid_search.fit(X_text_train, y_train)


    print(f"Classifier: LogisticRegression, Vectorizer: {vectorizer_name}")
    print("Best parameters found:")
    print(grid_search.best_params_)
    print("Best cross-validation f1:")
    print(grid_search.best_score_)
    print("")

    best_model_svm = grid_search.best_estimator_

    y_pred = best_model.predict(X_text_test)
    report = metrics.classification_report(y_test, y_pred)
    print("Classification Report:")
    print(report)
    print("")


Classifier: LogisticRegression, Vectorizer: CountVectorizer
Best parameters found:
{'classifier__C': 0.1, 'classifier__class_weight': None, 'classifier__degree': 2, 'classifier__gamma': 0.1, 'classifier__kernel': 'linear', 'classifier__probability': False, 'vectorizer__max_features': None, 'vectorizer__ngram_range': (1, 2), 'vectorizer__stop_words': None}
Best cross-validation f1:
0.828125

Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.82      0.81       193
           1       0.83      0.81      0.82       207

    accuracy                           0.81       400
   macro avg       0.81      0.81      0.81       400
weighted avg       0.81      0.81      0.81       400


Classifier: LogisticRegression, Vectorizer: TfidfVectorizer
Best parameters found:
{'classifier__C': 10, 'classifier__class_weight': None, 'classifier__degree': 2, 'classifier__gamma': 0.1, 'classifier__kernel': 'rbf', 'classifier__probability': False, 'v

In [103]:

pipe = Pipeline([
    ('vectorizer', CountVectorizer(max_features = None, ngram_range = (1, 2), stop_words = None)),
    ('classifier', SVC(C = 0.1, class_weight = None, degree = 2, gamma = 0.1, kernel = 'linear', probability = False))
])

pipe.fit(X_text_train, y_train)

y_pred = pipe.predict(X_text_test)
report = metrics.classification_report(y_test, y_pred)
print("Classification Report:")
print(report)
print("")

Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.84      0.83       194
           1       0.84      0.82      0.83       206

    accuracy                           0.83       400
   macro avg       0.83      0.83      0.83       400
weighted avg       0.83      0.83      0.83       400




In [114]:
y_pred = best_model_svm.predict(X_text_test)
report = metrics.classification_report(y_test, y_pred)
print("Classification Report:")
print(report)
print("")

Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.87      0.85       193
           1       0.87      0.83      0.85       207

    accuracy                           0.85       400
   macro avg       0.85      0.85      0.85       400
weighted avg       0.85      0.85      0.85       400




In [47]:
from sklearn import svm
from joblib import dump

dump(best_model_svm, 'svm_model_tfidf.joblib')

['svm_model_tfidf.joblib']

### Random forest


In [43]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizers = [
    ('CountVectorizer', CountVectorizer()),
    ('TfidfVectorizer', TfidfVectorizer())
]

param_grid = {
    'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'vectorizer__stop_words': [None, 'english'],
    'vectorizer__max_features': [1000, 5000, 10000, None],
    'classifier__n_estimators': [100, 200, 500],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]
}

for vectorizer_name, vectorizer in vectorizers:
    pipe = Pipeline([
        ('vectorizer', vectorizer),
        ('classifier', RandomForestClassifier())
    ])

    grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='f1_micro', n_jobs = -1, refit = True)  # Modifier les paramètres CV et scoring
    grid_search.fit(X_text_train, y_train)


    print(f"Classifier: LogisticRegression, Vectorizer: {vectorizer_name}")
    print("Best parameters found:")
    print(grid_search.best_params_)
    print("Best cross-validation f1:")
    print(grid_search.best_score_)
    print("")

    best_model_rf = grid_search.best_estimator_

    y_pred = best_model.predict(X_text_test)
    report = metrics.classification_report(y_test, y_pred)
    print("Classification Report:")
    print(report)
    print("")


Classifier: LogisticRegression, Vectorizer: CountVectorizer
Best parameters found:
{'classifier__max_depth': 10, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 10, 'classifier__n_estimators': 500, 'vectorizer__max_features': 10000, 'vectorizer__ngram_range': (1, 2), 'vectorizer__stop_words': 'english'}
Best cross-validation f1:
0.86

Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.82      0.81       193
           1       0.83      0.81      0.82       207

    accuracy                           0.81       400
   macro avg       0.81      0.81      0.81       400
weighted avg       0.81      0.81      0.81       400


Classifier: LogisticRegression, Vectorizer: TfidfVectorizer
Best parameters found:
{'classifier__max_depth': 10, 'classifier__min_samples_leaf': 4, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 500, 'vectorizer__max_features': 5000, 'vectorizer__ngram_range': (1, 2), 'vecto

In [39]:
pipe = Pipeline([
    ('vectorizer', CountVectorizer(max_features=5000, ngram_range=(1, 2), stop_words='english')),
    ('classifier', RandomForestClassifier(max_depth=10, min_samples_leaf=1, min_samples_split=10, n_estimators=500))
])

pipe.fit(X_text_train, y_train)

y_pred = pipe.predict(X_text_test)
report = metrics.classification_report(y_test, y_pred)
print("Classification Report:")
print(report)
print("")

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.86      0.86       194
           1       0.87      0.88      0.87       206

    accuracy                           0.87       400
   macro avg       0.87      0.87      0.87       400
weighted avg       0.87      0.87      0.87       400




In [45]:
pipe = Pipeline([
    ('vectorizer', TfidfVectorizer(max_features=5000, ngram_range=(1, 2), sublinear_tf=True )),
    ('classifier', RandomForestClassifier(max_depth=10, min_samples_leaf=1, min_samples_split=10, n_estimators=500))
])

pipe.fit(X_text_train, y_train)

y_pred = pipe.predict(X_text_test)
report = metrics.classification_report(y_test, y_pred)
print("Classification Report:")
print(report)
print("")

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.81      0.83       194
           1       0.83      0.86      0.85       206

    accuracy                           0.84       400
   macro avg       0.84      0.84      0.84       400
weighted avg       0.84      0.84      0.84       400




### Naive bayes bernoulli

In [44]:

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB
vectorizers = [
    ('CountVectorizer', CountVectorizer()),
    ('TfidfVectorizer', TfidfVectorizer())
]

param_grid = {
    'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'vectorizer__stop_words': [None, 'english'],
    'vectorizer__max_features': [1000, 5000, 10000, None],
    'classifier__alpha': [0.1, 1.0, 10.0],
    'classifier__binarize': [0.0, 0.5, 1.0]
}

for vectorizer_name, vectorizer in vectorizers:
    pipe = Pipeline([
        ('vectorizer', vectorizer),
        ('classifier', BernoulliNB())
    ])

    grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='f1_micro', n_jobs = -1, refit = True)  # Modifier les paramètres CV et scoring
    grid_search.fit(X_text_train, y_train)


    print(f"Classifier: LogisticRegression, Vectorizer: {vectorizer_name}")
    print("Best parameters found:")
    print(grid_search.best_params_)
    print("Best cross-validation f1:")
    print(grid_search.best_score_)
    print("")

    best_model_nb = grid_search.best_estimator_

    y_pred = best_model.predict(X_text_test)
    report = metrics.classification_report(y_test, y_pred)
    print("Classification Report:")
    print(report)
    print("")


Classifier: LogisticRegression, Vectorizer: CountVectorizer
Best parameters found:
{'classifier__alpha': 0.1, 'classifier__binarize': 0.0, 'vectorizer__max_features': 5000, 'vectorizer__ngram_range': (1, 3), 'vectorizer__stop_words': 'english'}
Best cross-validation f1:
0.819375

Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.82      0.81       193
           1       0.83      0.81      0.82       207

    accuracy                           0.81       400
   macro avg       0.81      0.81      0.81       400
weighted avg       0.81      0.81      0.81       400


Classifier: LogisticRegression, Vectorizer: TfidfVectorizer
Best parameters found:
{'classifier__alpha': 0.1, 'classifier__binarize': 0.0, 'vectorizer__max_features': 5000, 'vectorizer__ngram_range': (1, 3), 'vectorizer__stop_words': 'english'}
Best cross-validation f1:
0.819375

Classification Report:
              precision    recall  f1-score   support

         

In [80]:
pipe = Pipeline([
    ('vectorizer', CountVectorizer(max_features=5000, ngram_range=(1,3))),
    ('classifier', BernoulliNB())
])

pipe.fit(X_text_train, y_train)

y_pred = pipe.predict(X_text_test)
report = metrics.classification_report(y_test, y_pred)
print("Classification Report:")
print(report)
print("")

Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.82      0.80       194
           1       0.82      0.77      0.80       206

    accuracy                           0.80       400
   macro avg       0.80      0.80      0.80       400
weighted avg       0.80      0.80      0.80       400




In [79]:
pipe = Pipeline([
    ('vectorizer', TfidfVectorizer(max_features=5000, ngram_range=(1,3))),
    ('classifier', BernoulliNB())
])

pipe.fit(X_text_train, y_train)

y_pred = pipe.predict(X_text_test)
report = metrics.classification_report(y_test, y_pred)
print("Classification Report:")
print(report)
print("")

Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.82      0.80       194
           1       0.82      0.77      0.80       206

    accuracy                           0.80       400
   macro avg       0.80      0.80      0.80       400
weighted avg       0.80      0.80      0.80       400




###  SVD pour la regression logistique 

In [ ]:
X_text_train2, X_text_test2, y_train2, y_test2 = model_selection.train_test_split(preprocessed_movies_df['text'], preprocessed_movies_df['label'], test_size=0.2)

In [81]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import FunctionTransformer

# Créer une fonction personnalisée pour la normalisation MinMax
def min_max_normalize(X):
    X_normalized = (X - X.min()) / (X.max() - X.min())
    return X_normalized


pipe = Pipeline([
    ('vectorizer', TfidfVectorizer(sublinear_tf =  True)),
    ('normalize', FunctionTransformer(min_max_normalize, validate=False)),
    ('svd', TruncatedSVD(n_components=5000)),  # Ajout de l'étape SVD
    ('classifier', LogisticRegressionCV())
])

pipe.fit(X_text_train, y_train)


y_pred = pipe.predict(X_text_test)
report = metrics.classification_report(y_test, y_pred)
print("Classification Report:")
print(report)
print("")

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.86      0.88       194
           1       0.87      0.92      0.90       206

    accuracy                           0.89       400
   macro avg       0.89      0.89      0.89       400
weighted avg       0.89      0.89      0.89       400




In [96]:
fichier = open("./eval/testSentiment.txt", "r", encoding="utf-8")
contenu = fichier.read()
fichier.close()
eval_lines = contenu.split("\n")

In [97]:
preprocessed_eval_lines = [utils.preprocess(line) for line in eval_lines]
preprocessed_eval_lines = preprocessed_eval_lines[:-1]
preprocessed_eval_lines[-1]

'this is one of the dumbest films ive ever seen it rips off nearly ever type of thriller and manages to make a mess of them allbr br theres not a single good line or character in the whole mess if there was a plot it was an afterthought and as far as acting goes theres nothing good to say so ill say nothing i honestly cant understand how this type of nonsense gets produced and actually released does somebody somewhere not at some stage think oh my god this really is a load of shite and call it a day its crap like this that has people downloading illegally the trailer looks like a completely different film at least if you have download it you havent wasted your time or money dont waste your time this is painful'

In [98]:
y_eval_prob = pipe.predict(preprocessed_eval_lines)

with open("./eval_tfidfsvd.txt", "w", encoding="utf-8") as fichier_sortie:
    for line in y_eval_prob:

        prediction = "N" if line == 0 else "P"
        # Écrire la prédiction dans le fichier de sortie
        fichier_sortie.write(prediction + "\n")

In [99]:
fichier = open("./eval_predictions.txt", "r", encoding="utf-8")
contenu = fichier.read()
fichier.close()
TEST = contenu.split("\n")

In [100]:
len(eval_lines)

25001